In [1]:
import numpy as np
import pandas as pd
import ast


In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# print(credits['crew'])
movies= movies.merge(credits,on='title')
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

# print(movies.isnull().sum())
# print(movies.duplicated().sum())
# print(movies.iloc[0].genres)

import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [17]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 4:
            L.append(i['name'])
        counter+=1
    return L
movies['cast'] = movies['cast'].apply(convert3)

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L
movies['crew']=movies['crew'].apply(fetch_director)
movies['overview'] = movies['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [18]:

movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new_df=movies[['movie_id','title','tags']]

new_df = movies[['movie_id', 'title', 'tags']].copy()
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [19]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()


In [20]:
for i in cv.get_feature_names_out():
 print(i,end=', ')

000, 007, 10, 100, 11, 12, 13, 14, 15, 16, 17, 17th, 18, 18th, 19, 1930s, 1940s, 1950s, 1960s, 1970s, 1980, 1980s, 1985, 1990s, 19th, 19thcentury, 20, 200, 2009, 20th, 24, 25, 30, 300, 3d, 40, 50, 500, 60, 60s, 70, aaron, aaroneckhart, aarontaylor, abandoned, abducted, abigailbreslin, abilities, ability, able, aboard, abuse, abusive, academy, accept, accepted, accepts, access, accident, accidental, accidentally, accompanied, accomplish, account, accountant, accused, ace, achieve, act, acting, action, actionhero, actions, activist, activities, activity, actor, actors, actress, acts, actual, actually, adam, adambrody, adams, adamsandler, adamshankman, adaptation, adapted, addict, addicted, addiction, adolescence, adopt, adopted, adoption, adopts, adrienbrody, adult, adultery, adulthood, adults, advantage, adventure, adventures, advertising, advice, affair, affairs, affection, affections, afghanistan, africa, african, africanamerican, aftercreditsstinger, afterlife, aftermath, age, aged, 

In [21]:
import nltk
from nltk.stem.porter import PorterStemmer  

ps = PorterStemmer()

def stem(text):
    L = []
    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L)

In [22]:
new_df['tags'].apply(stem)

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4809, dtype: object

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector)
similarity[0][0]

In [ ]:
def recommend(movie):
    # Get index of the movie by title
    movie_index = new_df[new_df['title'] == movie].index[0]
    
    # Get similarity scores and sort them
    distances = list(enumerate(similarity[movie_index]))
    distances = sorted(distances, reverse=True, key=lambda x: x[1])
    print(type(distances))

    for i in distances[1:6]:
        print(new_df.iloc[i[0]]['title'])


In [ ]:
recommend('Gandhi')

In [ ]:
import pickle
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))